In [1]:
#https://ideepaksinghs.wordpress.com/2021/04/11/creating-a-self-driving-car-in-24-hour/
import sys
sys.path.append("..")
from carlasim.vehicle_hal import EgoCar
from carlasim.carla_client import CarlaClient
from carlasim.remote_controller import RemoteController
from carlasim.mqtt_client import MqttClient
from carlasim.frame_segment_converter_cuda import FrameSegmentConverterCuda
from carlasim.video_streamer import VideoStreamer
from planner.simple_slam import SimpleSlam
from planner.waypoint import Waypoint
import io

import gi
gi.require_version('Gst', '1.0')
gi.require_version('GstRtspServer', '1.0')
from gi.repository import Gst, GLib
Gst.init(None)

client = CarlaClient(town='Town07')
segment_converter = FrameSegmentConverterCuda()


In [2]:
car = EgoCar(client)\
    .with_bev_camera(20000, on_frame_callback=lambda f: car.send_bev_frame( segment_converter.convert_frame(VideoStreamer.to_rgb_array(f))))\
    .build()
car.set_pose(0, 0, 0)


In [3]:
car2 = EgoCar(client)\
    .with_bev_camera(-1, lambda f: None)\
    .set_color(0,255,0)\
    .build()
car2.set_pose(10, 0, 0)

car3 = EgoCar(client)\
    .with_bev_camera(-1, lambda f: None)\
    .set_color(255,0,0)\
    .build() 

car3.set_pose(0, 10, 0)

#https://carla.readthedocs.io/en/latest/tuto_G_bounding_boxes/

In [6]:
x = -25
y = 0

car.set_pose(x, y, 0)
car2.set_pose(x + SimpleSlam.DIST_FRONT, 0, 0)
car3.set_pose(x, y -SimpleSlam.DIST_LEFT, 0)


In [14]:

slam = SimpleSlam(400, 300, 0.0, 0.0)
location = car.get_location()
location2 = car2.get_location()

print(location)
print (location2)


goal = Waypoint(170, 0)

new_location = slam.estimate_next_pose(car, goal)
print(new_location)
car2.set_pose(new_location[0], new_location[1], new_location[2])




Location(x=-25.000002, y=0.000030, z=0.032080)
Location(x=-11.581527, y=3.013085, z=0.102168)
(-11.574951946481054, -2.979693226757429, -11.308866298963592)
